**Import required packages**

In [1]:
import pandas as pd
import json 
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import numpy as np
from numpy.random import dirichlet
from ema_workbench import TimeSeriesOutcome, Model, perform_experiments, RealParameter, Scenario, CategoricalParameter, ScalarOutcome, ema_logging, save_results, Policy
from ema_workbench.em_framework.samplers import LHSSampler  
from ema_workbench import SequentialEvaluator
from ema_workbench.analysis import lines, prim
from ema_workbench.analysis.plotting_util import Density
from ema_workbench.connectors.vensim import VensimModel

C:\Users\saski\anaconda3\Lib\site-packages\ema_workbench\connectors\__init__.py:29: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
C:\Users\saski\anaconda3\Lib\site-packages\ema_workbench\connectors\__init__.py:34: ImportWarning: simio connector not available
  warnings.warn("simio connector not available", ImportWarning)


In [2]:
#number of experiments
n_experiments = 1000 

In [3]:
#settings for market share per haul segment
num_segments = 5
lower_bound = 0.1 #minimum market share per haul segment = 10%
upper_bound = 0.3 #maximum market share per haul segment = 30%

**Define uncertain parameters**

In [4]:
uncertain_parameters = []
haul_segments = ["ShortHaul", "ShortMediumHaul", "MediumHaul", "LongHaul", "UltraLongHaul"]

for segment in haul_segments:
    uncertain_parameters.append(
            RealParameter(f"Market share per haul segment[{segment}]", lower_bound, upper_bound)
    )
        
    #add uncertain parameters for fare elasticity and pricing strategy factor
    uncertain_parameters.append(
        RealParameter(f"Fare elasticity of demand[{segment}]", -1.5, -0.6)
    )
    uncertain_parameters.append(
        RealParameter(f"Pricing strategy factor[{segment}]", 0.1, 1.0)
    )

**Define the Market Share Sampling**

Sample market shares per haul segment, ensuring them to meet the sum and bounds constraints.
Since "Market share per haul segment" is subscripted, Dirichlet-based sampling is used. 

In [5]:
np.random.seed(8) #set random seed to ensure reproducibility

def sample_market_shares(num_segments, lower_bound, upper_bound, n_experiments):
    
    all_market_shares = []
    
    for _ in range(n_experiments):
        alpha = np.ones(num_segments) #equal weights for haul segments
        shares = np.random.dirichlet(alpha)
    
        scaled_shares = lower_bound + shares * (upper_bound - lower_bound) #scale shares to fit within lower and upper bound
    
        normalized_shares = scaled_shares / scaled_shares.sum() #make sure that sum of shares is always equal to 1
    
        #check whether the sum of the shares is indeed 1
        #if np.isclose(normalized_shares.sum(), 1):
            #print(f"Market shares sum to 1: {normalized_shares.sum()}")  
        #else:
            #print(f"Warning: Market shares do not sum to 1. Sum: {normalized_shares.sum()}")
            
        all_market_shares.append(normalized_shares)
    
    return np.array(all_market_shares)

market_shares_for_all_runs = sample_market_shares(num_segments, lower_bound, upper_bound, n_experiments)

#print("Market Shares for Each Experiment:")
#for i, shares in enumerate(market_shares_for_all_runs):
    #print(f"Run {i+1}: {shares}")
    
scenarios = []

for i in range(n_experiments):
    scenario = Scenario()
    
    # Add market share values for each segment
    for j, segment in enumerate(haul_segments):
        scenario[f"Market share per haul segment[{segment}]"] = market_shares_for_all_runs[i, j]
        
    for segment in haul_segments:
        scenario[f"Fare elasticity of demand[{segment}]"] = None  # LHS will handle this
        scenario[f"Pricing strategy factor[{segment}]"] = None  # LHS will handle this
    
    scenarios.append(scenario)

**Define the Vensim Model**

In [6]:
# turn on logging
ema_logging.log_to_stderr(ema_logging.INFO)

# Initialize the Vensim model
#wd = r"C:\Users\saski\OneDrive\Documenten\EPA\AFSTUDEREN\Model\Definitive Model Version"
#vensimModel = VensimModel("FuturePredictionModel", wd=wd, model_file="Model Future Prediction 17.1 FINAL.vpmx")
model_directory2 = r"C:\Users\saski\OneDrive\Documenten\EPA\AFSTUDEREN\Model\Definitive Model Version"
model_file2 = "final_model_flightrestriction.vpmx"
vensim_model2 = VensimModel("model2", wd=model_directory2, model_file=model_file2)
    
vensim_model2.sampler = LHSSampler() #use LHS sampling instead of random sampling

**Define uncertainties**

In [7]:
vensim_model2.uncertainties = uncertain_parameters

**Define KPIs**

In [8]:
vensim_model2.outcomes = [TimeSeriesOutcome("Total accumulated emissions", variable_name="Total accumulated emissions"),
                         TimeSeriesOutcome("Total annual passenger demand", variable_name="Total annual passenger demand"),
                         TimeSeriesOutcome("Average demand fulfillment rate", variable_name="Average demand fulfillment rate")]
                         #TimeSeriesOutcome("Change ratio fuel cost per RPK", variable_name="Change ratio fuel cost per RPK"),
                         #TimeSeriesOutcome("Fare price change ratio", variable_name="Fare price change ratio"),
                         #TimeSeriesOutcome("Fuel efficiency-induced demand growth rate", variable_name="Fuel efficiency induced demand growth rate")]

**Run experiments**

In [10]:
from ema_workbench import SequentialEvaluator

experiments2, outcomes2 = perform_experiments(vensim_model2, scenarios=n_experiments)

[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
100%|██████████████████████████████████████| 1000/1000 [01:01<00:00, 16.30it/s]
[MainProcess/INFO] experiments finished


In [11]:
save_results((experiments2, outcomes2), "./results/results_final_flightrestriction.tar.gz")

C:\Users\saski\anaconda3\Lib\site-packages\ema_workbench\em_framework\outcomes.py:541: UserWarning: still to be tested!!
  warnings.warn("still to be tested!!")
[MainProcess/INFO] results saved successfully to C:\Users\saski\OneDrive\Documenten\EPA\AFSTUDEREN\results\results_final_flightrestriction.tar.gz
